<a href="https://colab.research.google.com/github/lmoss/onesharp/blob/main/primitive_recursion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Primitive Recursion

This notebook enables one to enter primitive recursive definitions and convert them to 1#.

It is mainly under development, but one could play with it now.

To start, enter all the code cells below and then look at the definitions below.


In [2]:
#@title
import pandas as pd
import numpy as np

def ones(n):
    w = ['1' for i in range(n)]
    return(unparse(w))    


def program_checker(str):
    m = len(str)
    x1 = str[m - 1] == '#'
    x2 = all((str[i] == '1' or str[i] == '#') for i in range(m))
    x3 = (str.find('######') == -1)
    if (x1 and x2 and x3):
        flag = True
    else:
        flag = False
        print('The input ' + str + ' is not a valid 1# program.')
        print('It is not the concatenation of a sequence of instructions in the language.')
        print('So what you are asking for is undefined.')
    return (flag)

def one_or_sharp_check(letter):
    if (letter=="1" or letter=="#"):
        return(True)
    else:
        return(False)

def word_checker(strg):
    answer = all([one_or_sharp_check(x)==True for x in strg])
    return(answer)

def input_checker(input_seq):
    seq = [word_checker(x) for x in input_seq]
    flag = all([word_checker(x) for x in input_seq])
    if not flag:
        print('The input sequence contains words with characters other than 1 and #.')
        print('So what you are asking for is undefined.')
    return(flag)
                                                                                
class Augmented:
    def __init__(self, string, remainders):
        self.string = string
        self.remainders = remainders

class Snapshot:
    def __init__(self, instr_number, regs, proceed,verbose,program_length, step_number):
        self.instr_number = instr_number
        self.regs = regs
        self.proceed = proceed
        self.verbose = verbose
        self.program_length = program_length
        self.step_number = step_number
        
def preparse(xstr):
    b = xstr.string.find('#1')
    xstr.remainders = xstr.remainders + [xstr.string[:(b + 1)]]
    xstr.string = xstr.string[(b + 1):]
    return (xstr)



def parse(y):
    tempx = Augmented(y, [])
    while tempx.string.find('#1') >= 0:
        tempx = preparse(tempx)
    return (tempx.remainders + [tempx.string])

def unparse(p):
    return (''.join(p))

def instruction_type(instruction):
    if instruction[-2:] == '1#':
        return ('add1')
    if instruction[-3:] == '1##':
        return ('add#')
    if instruction[-4:] == '1###':
        return ('forward')
    if instruction[-5:] == '1####':
        return ('backward')
    if instruction[-6:] == '1#####':
        return ('cases')

def tail(list):
    return (list[1:])

def one_step(p, snapshot): # p is parsed
    i = snapshot.instr_number
    r = snapshot.regs
    instruction = p[-1 + i]
    if snapshot.verbose:
        print('Step ' + str(snapshot.step_number) + ':')
        print('Execute instruction ' + str(i) + ':' + " " +
              instruction_gloss(instruction,i-1) 
              + '.')
        if instruction_type(instruction)=='cases':
            billy= len(instruction) - 5
            if snapshot.regs[billy-1] == "":
                print('The register is empty, so we go ahead 1 instruction.')
            elif snapshot.regs[billy-1][0] == "1":
                print('The first symbol in that register is 1,' +
                      ' so we delete that symbol and go forward 2 instructions.')
            elif snapshot.regs[billy-1][0] == "#":
                print('The first symbol in that register is #,' +
                      ' so we delete that symbol and go forward 3 instructions.')     
    t = instruction_type(instruction)
    if t == 'add1':
        snapshot.instr_number = 1 + snapshot.instr_number
        l = len(instruction)
        reg = len(instruction[:(l - 1)])
        snapshot.regs[reg - 1] = snapshot.regs[reg - 1] + '1'
    if t == 'add#':
        snapshot.instr_number = 1 + snapshot.instr_number
        l = len(instruction)
        reg = len(instruction[:(l - 2)])
        snapshot.regs[reg - 1] = snapshot.regs[reg - 1] + '#'
    if t == 'forward':
        l = len(instruction)
        offset = len(instruction[:(l - 3)])
        snapshot.instr_number = offset + snapshot.instr_number
    if t == 'backward':
        l = len(instruction)
        offset = len(instruction[:(l - 4)])
        snapshot.instr_number = (-offset) + snapshot.instr_number
    if t == 'cases':
        l = len(instruction)
        reg = len(instruction[:(l - 5)])
        if snapshot.regs[reg - 1] == '':
            snapshot.instr_number = 1 + snapshot.instr_number
        elif snapshot.regs[reg - 1][0] == '1':
            snapshot.instr_number = 2 + snapshot.instr_number
            snapshot.regs[reg - 1] = tail(snapshot.regs[reg - 1])
        elif snapshot.regs[reg - 1][0] == '#':
            snapshot.instr_number = 3 + snapshot.instr_number
            snapshot.regs[reg - 1] = tail(snapshot.regs[reg - 1])
    if 0< snapshot.instr_number <= len(p):
        snapshot.proceed = True
        if snapshot.verbose == True:
            print_snapshot(snapshot)
    else:
         snapshot.proceed = False
    return (snapshot)


def number_help(instr):
    if instruction_type(instr) == 'add1':
        return (len(instr) - 1)
    if instruction_type(instr) == 'add#':
        return (len(instr) - 2)
    if instruction_type(instr) == 'cases':
        return (len(instr)-5)
    else:
        return (0)


def max_register(p):
    return (max([number_help(instr) for instr in parse(p)]))


def pad(p, register_inputs):
    n = len(register_inputs)
    m = max_register(p)
    extras = ['' for x in range(m - n)]
    bigger = register_inputs + extras
    return (bigger)

def print_snapshot(snap):
    regdf = pd.DataFrame([[snap.regs[n]] for n in range(len(snap.regs))],columns=["contents"])
    regdf.index = np.arange(1, len(regdf) + 1)
    def make_pretty(styler):
        styler.set_properties(**{'background-color': '#FFFFCC'})
        styler.set_properties(**{'text-align': 'left'})
        #styler.set_caption("at the start")
        #styler.hide(axis='index')
        return styler
    display(regdf.style.pipe(make_pretty))  

def step_by_step(word_prog, register_inputs):
    word_prog = word_prog.replace(" ", "")
    register_inputs = [word.replace(" ", "") for word in register_inputs]
    if program_checker(word_prog) and input_checker(register_inputs):
        print('First, here is the program:')
        parse_explain(word_prog)
        print()
        regs = pad(word_prog, register_inputs)
        prog = parse(word_prog)
        N = len(prog)
        snap = Snapshot(1, regs,True,True,N,1)
        print('The computation starts with the register contents shown below.')
        print('The registers include those those which you entered as part of the input')
        print('and also others mentioned in the input program.')
        print_snapshot(snap)
        print()
        while 0 < snap.instr_number < N + 1:
            snap = one_step(prog, snap)
            snap.step_number = (snap.step_number) + 1
        if snap.instr_number <= 0:
            print(
                'The computation has not halted properly ' +
                'because the control went above instruction 1 of the program.'
                 )
        elif (snap.instr_number == (N + 1)) and all(
                snap.regs[i] == ""
                for i in range(1, len(snap.regs))):
            print(
                'The computation then halts properly because' +
                ' the control is just below the last line of the program,')
            print('and because all registers other than R1 are empty.')
            if snap.regs[0] == "":
                print('The output is the empty word.')
            else:
                print('The output is ' + snap.regs[0] + '.')
        else:
            print('This computation does not halt.')
            if snap.instr_number != N + 1:
                print('This is because the program has ' + str(len(prog)) +
                  ' instructions, and control at the end is not one line ' + 
                   'below the bottom of the program.')
                print()
            else:
                not_blank = [
                    i + 1 for i in range(1, len(snap.regs))
                    if snap.regs[i] != ""
                ]
                print('Here is the list of registers whose contents ' +
                      'are not empty at this point, other than R1:' +
                      str(not_blank) + '.')
                print('The register contents at the end are shown above.')


def onesharp(word_prog, register_inputs):
    word_prog = word_prog.replace(" ", "")
    register_inputs = [word.replace(" ", "") for word in register_inputs]  
    if program_checker(word_prog) and input_checker(register_inputs):
        register_inputs = [word.replace(" ", "") for word in register_inputs]
        regs = pad(word_prog, register_inputs)
        prog = parse(word_prog)
        N = len(prog)
        snap = Snapshot(1, regs,True,False, N, 1)
        while snap.proceed:
            snap = one_step(prog, snap)
            snap.step_number = (snap.step_number)+1
        if (snap.instr_number == (N + 1)) and all(
                snap.regs[i] == "" for i in range(1, len(snap.regs))):
            return ((snap.regs)[0])
        else:            
            print("This is undefined.")
            print("The register contents at the end are shown below.")
            print_snapshot(snap)
    else:
        return('undefined')


def instruction_gloss(instr,line):
    if instruction_type(instr) == 'add1':
        return ('add 1 to R' + str(len(instr) - 1))
    if instruction_type(instr) == 'add#':
        return ('add # to R' + str(len(instr) - 2))
    if instruction_type(instr) == 'forward':
        w = len(instr) - 3
        return ('go forward ' + str(w) + ' to instruction ' + str(w+line+1))
    if instruction_type(instr) == 'backward':
        w = len(instr) - 4
        return ('go backward ' + str(w) + ' to instruction ' + str(line - w+1))
    if instruction_type(instr) == 'cases':
        return ('cases on R' + str(len(instr) - 5))

def expanded(gorp):
    pgorp = parse(gorp)
    wwgorp = [[pgorp[x],instruction_gloss(pgorp[x],x)] for x in range(len(pgorp))]
    return(wwgorp)

def parse_explain(prog):
    df = pd.DataFrame(expanded(prog),
                      columns=["instruction", 'explanation'])
    df.index = np.arange(1, len(df) + 1)
    def make_pretty(styler):
                styler.set_properties(**{'background-color': '#C9DFEC'})        
                styler.set_properties(**{'text-align': 'left'})
                return styler
    display(df.style.pipe(make_pretty))
    #display(df)
    
clear_1 = '1#####111###11####111####'

move_2_1 = '11#####111111###111###1##1111####1#111111####'

copy_1_2_3 = '1#####11111111###1111###11##111##11111####11#111#11111111####111#####111111###111###1##1111####1#111111####'

length = '1#####1111111###11####11#1#####111###111111####111####11#####111111###111###1##1111####1#111111####'

write = '1#####111111111###11111###11#11##11##111111####11#11##111111111####11#####111111###111###1##1111####1#111111####'

diag = '1#####11111111111###111111###11##111#111##111##1111111####11#111#111##1111####111#####111111###111###1##1111####1#11####11#####111111###111###1##1111####1#11####'

self = '1#1##1##1##1##1##1#1#1#1#1#1#1#1#1#1#1#1##1##1##1#1#1#1#1#1#1##1##1##1#1#1##1##1#1#1#1##1#1#1#1##1##1#1#1#1##1##1#1#1#1#1#1#1#1##1##1##1##1#1#1##1#1#1#1##1#1#1#1##1##1#1#1#1#1##1##1##1##1#1#1#1##1##1##1##1##1#1#1#1#1#1#1##1##1##1#1#1#1##1##1##1#1##1##1#1#1#1#1##1##1##1##1#1##1#1#1##1##1##1##1#1#1##1##1##1##1##1#1#1#1#1#1#1##1##1##1#1#1#1##1##1##1#1##1##1#1#1#1#1##1##1##1##1#1##1#1#1##1##1##1##1#####11111111111###111111###11##111#111##111##1111111####11#111#111##1111####111#####111111###111###1##1111####1#11####11#####111111###111###1##1111####1#11####'

multiply = '111##1111##11#####11111111###1111###11111##111111##11111####11111#111111#11111111####111111#####111111###111###11##1111####11#111111####111#####11111111###1111###111111##1111111##11111####111111#1111111#11111111####1111111#####111111###111###111##1111####111#111111####11111#####111111###111111111###111111#####11111111111###1111111111###111111####111111#####111111111111111###111111###11111###111111#####111###1111111111111####1###11111#####111###11####111####111111#####1111###11####111####11111#11111#####111###1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111###1#1#####11111111###1111###11111##111111##11111####11111#111111#11111111####111111#####111111###111###1##1111####1#111111####1111#####111111###111###111111##1111####111111#111111####11111#####111###111111###111111111###111111#####11111111111111111111111111111111111###11111111111111111111111111###111111111111111111111111111###111111#####11111111111111111111111###1111111111111111111111111111###111111111111111111111###111111#####111111111111111111111###111111111111111111###1111111111111111111###11111#####111###111111###111111111###111111#####11111111111###1111111111111111###111111111###111111#####1111111111111###1111111111###11111111111###111111#####111###11111111###1###1111111#111111111111111111111111111111111####1111111##11111111111111111111111111111111111####1111111#111111111111111111111####1111111##11111111111111111111111####1###1111111#####111111###111###11111##1111####11111#111111####11111#####111111###111###1111##1111####1111#111111####111#####1111111111111###1111111111###11111#111#####111111###111###11111##1111####11111#111111####1111###11111##1111111111111####11111#11111#####111111###111###111##1111####111#111111####1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111####1#####111###11####111####11#####111###11####111####111#####111###11####111####1111#####111111###111###1##1111####1#111111####'

universal = '1#####1###11###11####111111#1#####111###111111###1111111###11111#####1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111###1###1####111#111111111####111##1#####1111###11###11111###111#11111#####1111111111111111111111111111111111111111111111###111111111###111##1#####1111###11###11111###111#11111#####1111111111111111111111111111111111111###111111111###111##1#####1111###11###11111###111#11111#####1111111111111111111111111111111111111111111111###111111111###111##1#####1111###11###11111###111#11111#####111111111111111111111111111111111111111111111111111111###111111111###111##1#####1111###11###11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111###111#11111#####1111111111###1###111#####111111###111###1111##1111####1111#111111####11111111111111111111111111111111111111111111111111111111111111####111#####1###11###1111###11111#1111#111111####11111##1111##111#####1111111###1111###1111##11111##11111####1111#1111111####111111111111111111111111111111111111111111111111111111111111111111111111111###111#####1###11###11111###11111#1111#111111#1111111####1111##111#####111111###111###1111##1111####1111#111111####11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111####111#####1###11###1111###1111#11111#111111####11111#11111#1111##111#####111111###111###1111##1111####1111#111111####11111#####11111111###11###1###111111#####111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111###1###1###11111111####111111#####1111###111#11111#1111####111#####111###111111#111####1#####111111###111###111##1111####111#111111####1111#####111111###111###1##1111####1#111111####111#####111111###111###1##1111####1#111111####111111111111111111111111111111111111111111111111111111111####11#####111###11111###111111111111###11##11##111111####111#11#####1###111###111##11###111#11111111111111####111##11#####11111111111111111111111111111111111111111111111111111111111111111111###1###111##11111#####1###1111111111111111111111####11111#####1111111111111111111111111111###111111111111111111111111111###11111#####111###11###111111111111111111111111111111111111111111111###11#####111111111111111111###111111111###111#11#####1###1###111#111##111##111111111111111111111111111111111111111111111111111111111###111#11#####1###111###111##11###111#111111111111111111####111#111#1111111111111111111111111111111111111111111111###11#####111111111111111111###111111111###111#11#####1###1###111##111##111##11111111111111111111111111111111111###111#11#####1###111###111##11###111#111111111111111111####111#111##111111111111111111111111###11111#####1###1###11111#####1###1###11#####11111111111111111111111###111###111##1111111111111###11#####1###11111###1###11111#111111#111111###11111#11111#111111#111111#1###11#####111111###111###111##1111####111#111111####111#####111111###111###11##1111####11#111111####1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111####11#####1###1###11#####1###1###11#####11111111111111###11###11111111###11#####1###111###1#11111111####1##1111111111####11#####111###11####111####111111#####11###11####1111#####111###11####111####'

clear_2 = '11#####111###11####111####'
clear_3 = '111#####111###11####111####'
clear_4 = '1111#####111###11####111####'
move_1_2= '1#####111111###111###11##1111####11#111111####'
move_1_3= '1#####111111###111###111##1111####111#111111####'
move_1_4= '1#####111111###111###1111##1111####1111#111111####'
move_2_1= '11#####111111###111###1##1111####1#111111####'
move_2_3= '11#####111111###111###111##1111####111#111111####'
move_2_4= '11#####111111###111###1111##1111####1111#111111####'
move_3_1= '111#####111111###111###1##1111####1#111111####'
move_3_2= '111#####111111###111###11##1111####11#111111####'
move_3_4= '111#####111111###111###1111##1111####1111#111111####'
move_4_1= '1111#####111111###111###1##1111####1#111111####'
move_4_2= '1111#####111111###111###11##1111####11#111111####'
move_4_3= '1111#####111111###111###111##1111####111#111111####'
copy_1_2_3 = '1#####11111111###1111###11##111##11111####11#111#11111111####111#####111111###111###1##1111####1#111111####'
copy_1_2_4 ='1#####11111111###1111###11##1111##11111####11#1111#11111111####1111#####111111###111###1##1111####1#111111####'
copy_1_3_4='1#####11111111###1111###111##1111##11111####111#1111#11111111####1111#####111111###111###1##1111####1#111111####'
copy_2_3_4 = '11#####11111111###1111###111##1111##11111####111#1111#11111111####1111#####111111###111###11##1111####11#111111####'

Sanity code below

In [3]:
#@title
def findme(pair,pair_list): 
  x = [i for i in range(len(pair_list)) if pair_list[i][0]==pair[1]]
  return x[0]

def add_placeholder(line):
  keywords = ['cases','add1','add#','goto','non_label']
  if line[0] in keywords:
    return(['non_label']+line)
  elif line[0][0] in ['1','#']:
    return(['non_label']+line)  
  else:
    return(line)

def flatten(tuple):
  b = len(tuple)
  if b > 1 and tuple[1] == "cases":
    n = tuple[2]
    x = ones(n)+'#####'
    return [[tuple[0],x], [tuple[0]+'@1',tuple[3]], [tuple[0]+'@2',tuple[4]], [tuple[0]+'@3',tuple[5]]]
  elif b > 1 and tuple[1]=='add1':
    return([[tuple[0],ones(tuple[2])+'#']])
  elif b > 1 and tuple[1]=='add#':
    return([[tuple[0],ones(tuple[2])+'##']])
  elif b > 1 and tuple[1]=='goto':
    return [[tuple[0],tuple[2]]]
  elif tuple[1][0] in ['1', '#']:
    k = parse(tuple[1])
    m = len(k)
    return([[tuple[0],k[j]] for j in range(m)])


def ones(n):
  return(unparse(['1' for i in range(n)])) 

def resolve(index,pair_list):
  pair_in_list = pair_list[index]
  #print("pair_in_list " + str(pair_in_list))
  first_char = pair_in_list[1][0]
  #print(first_char)
  if first_char == '1' or first_char == '#':
    return pair_in_list[1]
  elif pair_in_list[1]=='end':
    n = len(pair_list) - index
    return(ones(n)+'###')
  else:
    k = findme(pair_in_list,pair_list)
    if k > index:
      return(ones(k-index)+'###')
    if k < index:
      return(ones(index-k)+'####')


def sanity(line_list):
  w = [add_placeholder(line) for line in line_list]
  #print("[add_placeholder(line) for line in line_list] is")
  #print(w)
  s1 = [flatten(line) for line in w]
  #print('[flatten(line) for line in w] is')
  #print(s1)
  t1 = [item for sublist in s1 for item in sublist]
  #print('[item for sublist in s1 for item in sublist] is')
  #print(t1)
  n = len(t1)
  #print('len of t1 = ' + str(n))
  u1 = [resolve(i,t1) for i in range(n)] 
  return(unparse(u1))      

The next cell contains a few 1# programs special to this notebook.  

In [4]:
ddd = '1#####111111###111111111###11#####11111111111###1111111111###111111####11#####111111111111111###111111###11111###11#####111###1111111111111####1###1#####111###11####111####11#####1111###11####111####1#'

def compare_prog(a,b): #uses a+1, a+2, a+3
  p = copy_prog(a,a+1,a+2) + copy_prog(b,a+2,a+3) +  bump(ddd,a) 
  return(p)
  # checks if Register a and Register b have the same thing, 
  # preserving them.   At the end, R(a+1) either has 
  # 1 (if original Rn and Rm are the same) or is empty (if they aren't)
  # This program also writes to R(a+2) and R(a+3).  So when it is 
  # called, all of registers a+1, a+2, and a+3 should be empty

def successor_prog(to_increment,to_use):
  # gives a program to increment the register 'to_increment' by 1, using 
  # the register 'to_use'
   p = (ones(to_increment) + 
     '#####' + 
     '111###' +
     '111###' +
     '1111111111111111111111111###'+
     '1111111111111111111111111111111111111111###' +
     ones(to_use) + '##' + 
     ones(to_increment) + '#####' + 
     '111###' +
     '111111111111111111###' +
     '1111111111111111111###' +
     ones(to_use) + '#' + 
     ones(to_increment) + '#####' + 
     '111111###' +
     '111###' +
     ones(to_use) + '##' + 
     '1111####' +
     ones(to_use) + '#' + 
     '111111####' +
     ones(to_use) + '#####' + 
     '111111###' +
     '111###' +
     ones(to_increment) + '##' + 
     '1111####' +
     ones(to_increment) + '#' +  
     '111111####' +
     '1111111111111111111###'  +
     ones(to_use) + '##' +  
     '111111111111111111111####' +
     ones(to_use) + '#' +  
     ones(to_increment) + '#####' +  
     '111111###' +
     '111###' +
     ones(to_use) + '##' +  
     '1111####' +
     ones(to_use) + '#' + 
     '111111####' +
     ones(to_use) + '#####' +  
     '111111###' +
     '111###' +
     ones(to_increment) + '##' +  
     '1111####' +
     ones(to_increment) + '#' + 
     '111111####' +
     '1###'
     )
   return(p) 
  

More functions specific to this notebook.

In [10]:
z_prog = '11##'
one_prog = '11#'
#@title
# s is an index of the successor function in bb
successor = '1#####111###111###1111111111111111111111111###1111111111111111111111111111111111111111###11##1#####111###111111111111111111###1111111111111111111###11#1#####111111###111###11##1111####11#111111####11#####111111###111###1##1111####1#111111####1111111111111111111###11##111111111111111111111####11#1#####111111###111###11##1111####11#111111####11#####111111###111###1##1111####1#111111####1###'

def maxie(l):
  if l == []:
    return(0)
  else:
    return(max(l))
def height(tree):
  n = maxie([height(u) +1 for u in tree.children])
  return(n)
def all_equal(list):
  if (len(list)==0 or len(list)==1):
    return(True)
  else:
    b = all_equal(list[1:])  
    if b == False:
      return(False)
    else:
      return(list[0]==list[1])

def bump_instr(inst, amount):
  if instruction_type(inst) in ['forward','backward']:  
    return(inst) 
  elif instruction_type(inst) == 'add1':  
    n = number_help(inst)
    return(ones(n+amount)+'#')
  elif instruction_type(inst) == 'add#':  
    n = number_help(inst)
    return(ones(n+amount)+'##')
  elif instruction_type(inst) == 'cases':  
    n = number_help(inst)
    return(ones(n+amount)+'#####')
def bump(prog,amount):
  par = parse(prog)
  t = [bump_instr(instr,amount) for instr in par]
  return(unparse(t))

def clear_prog(n):
   a = ones(n) + '#####'
   b = '111###'
   c = '1###'
   d = '111####'
   return(a+b+c+d)

def move_prog(n,m):
   a = ones(n) + '#####'
   b = '111111###'
   c = '111###'
   d = ones(m)+'##'
   e = '1111####'
   f = ones(m)+'#'
   g = '111111####'
   return(a+b+c+d+e+f+g)

def copy_prog(n,m,p):
   a = ones(n) + '#####'
   b = '11111111###'
   c = '1111###'
   d = ones(m)+'##'
   d1 = ones(p) + '##'
   e = '11111####'
   f = ones(m)+'#'
   f1 = ones(p) + '#'
   g = '11111111####'
   return(a+b+c+d+d1+e+f+f1+g+move_prog(p,n))  

def proj_prog_official(ind,upper):
  index = [k+1 for k in range(ind-1)]    
  #print(index)
  first_part = [clear_prog(j) for j in index]
  #print(first_part)
  if ind == 1:
    middle = []
  else: 
    middle=[move_prog(ind,1)]
  second_index =  [k+ind+1 for k in range(upper-ind)] 
  #print(second_index)
  last_part = [clear_prog(j) for j in second_index]   
  together = first_part + middle + last_part
  done = unparse(together)
  if ind == 1 and upper==1:
    return('1###')
  else:
    return(done)   

def proj_prog(ind,upper):
  return(copy_prog(ind,upper+1, upper+2))

# the function below doesn't seem to be used
def copy_all_forward(n,m):
  # copies 1 to m, 2 to m+1, . . . n to m+n-1
  a = [copy_prog(i,m+i-1, m+i) for i in range(1,n+1)]
  return(unparse(a))


def compose(outer_fn,proglist,argument_number,used_reg):
  ## note that the outer_fn has to be 'in place'!
  k = len(proglist) # number of functions
  # the outer_fn is k-ary
  q = unparse([clear_prog(i) for i in range(1,k+1)])
  outer_fn = outer_fn + q + move_prog(k+1,1)
  arg = argument_number # = common arity in the proglist
  #max_list =   [max_register(p) for p in proglist]
  #m = maxie(max_list)
  free_reg = used_reg + 1
  a = [proglist[i] + move_prog(arg+1,free_reg+i+1) for i in range(0,k)]
  b = unparse(a)
  c = [move_prog(free_reg+i+1,argument_number+i+1) for i in range(0,k)]
  d = unparse(c)
  f = bump(outer_fn,argument_number)
  e = b + d + f   
  return(e)

# <g,h>(x-bar,0) = g(x-bar)
# <g,h>(x-bar,n+1) = h(x-bar,n,<g,h>(x-bar,n))
# k-1 is the number of xs, so the arity of <g,h> is k
# k+1 is the number of arguments to g
# k is the number of arguments to the function we are building
#free is large enough so that neither g nor h use it or any larger register.
# g and h are assumed to preserve their inputs
def primitive_recursion(g,h,arity,used_reg):
  free = used_reg +1
  free_plus_one = free + 1
  plan = [['top',move_prog(arity,free) +
               g + 
               move_prog(arity,arity+1) + 
               ones(arity) + '##' ],
          [compare_prog(free,arity)],
          ['decision', 'cases', free_plus_one, 'main_loop','clear_out_free', 'end' ],
          ['clear_out_free', clear_prog(free)],
          ['goto', 'end'],
          ['main_loop',h + 
               clear_prog(arity+1) + 
               move_prog(arity+2,arity+1) + 
               successor_prog(arity,arity+2) +
               compare_prog(free,arity)],
          ['goto', 'decision'],
        ]
  trial = sanity(plan)
  return(trial)  

In [6]:

def all_equal(list):
  a = list[0]
  b = [x == a for x in list]
  return(all(b))


def arity_check(t):
  if t[0] == 'proj':
    b = (isinstance(t[1],int) and isinstance(t[2],int) and (t[1] <= t[2]) and
        len(t)==3)
    return(b)

def arity(t):
  if t == 's_prog':
    return(1)
  if t[0] == 'proj':
    return(t[2])
  if t == 'z':
    return(1)
  if t == 'z_empty':
    return(0)
  if t[0] == 'pr':
    k = arity(t[1])
    j = arity(t[2])
    if j== 2 + k:
      return(k+1)
  if t[0] == 'comp':
    f = t[1]
    g_list = t[2:]
    if arity(f)== len(g_list) and all_equal([arity(v) for v in g_list]):
      return(arity(t[2]))   

def program(t):
  if t=='s_prog':
    return(copy_1_2_3+ successor_prog(2,3))
  if t[0] == 'proj':
    return(proj_prog(t[1],t[2]))
  if t == 'z':
    return('11##')
  if t == 'z_empty':
    return('1##') 
  if t[0] == 'pr':
    f = program(t[1])
    g = program(t[2])
    p = primitive_recursion(f, g, arity(t),essentially_used(t))
    return(p)
  if t[0] == 'comp':       
    g = program(t[1])
    hs = [program(u) for u in t[2:]]
    k = arity(t[2])
    p = compose(g,hs,k, essentially_used(t))
    return(p)

def essentially_used(t):
    if t=='s_prog':
      return(3)
    if t[0]== 'proj':
      return(t[2])
    if t == 'z':
      return(1)
    if t == 'z_empty':
      return(0)
    if t[0] == 'comp':
      outer = t[1]
      n1 = essentially_used(outer) 
      n2 = n1 + (len(t) -0)
      n3 = max([essentially_used(u) for u in t[2:]])
      n = max([n1,n2])
      return(n)
    if t[0] == 'pr':
      g = t[1]
      h = t[2]     
      n1 = essentially_used(g)
      n2 = essentially_used(h)
      n = max([n1,n2])
      return(n)

pr = 'pr'
proj = 'proj'
comp = 'comp'
s = 's_prog'
z = 'z'
z_empty = 'z_empty'
   


# The examples of primitive recursive definitions are just below.

Please note that the is strict; there are no meaningful error messages yet.

The base primitive recursive functons are 

z : the one-place function with value 0
z_empty: the zero-place function with value 0
[pr, i, j] with i <= j: this is the ith projection on j variables.

The constructors are

[comp, f, g_1, . . . , g_k]

[pr, f, g]

What we are constructing here are _primitive recursive terms_.  You can assign names to them, as shown below. The main functions in the notebook are

arity(t)

program(t)

#### Special note: the 1# programs in this notebook do not halt in the sense of the course.  Instead, they _preserve their inputs_.  For example, run the program associated with _add_ below as shown below it.  (That is, take away the # from the beginning of the lines after the definition. The 2 inputs are preserved and the result goes in register 3.)


In [11]:
add = [pr,[proj,1,1], [comp, s, [proj, 3,3]]]
#onesharp(program(add), ['11','#1#1'])

mul = [pr, z, [comp, add,[proj, 3,3],[proj, 1, 3]] ]
q = program(mul)
#onesharp(q, ['11','1#1'])

exp = [pr, [comp, s, z], [comp, mul, [proj, 3,3],[proj, 1, 3]] ]
pred = [pr, z_empty, [proj,1,2]]
monus = [pr, [proj, 1,1], [comp, pred, [proj,3,3]]]

two_place_fn_value_one = [comp, s,[comp,z,[proj,1,2]]]  
zero_place_fn_value_one = [comp, s, z_empty]  

sgn =[pr, z_empty,two_place_fn_value_one]
sgn_bar =[pr, zero_place_fn_value_one, [comp,z,[proj,1,2]]]

chi_greater = [comp,sgn, monus] ## characteristic function of >
chi_lesser_or_equal = [comp,sgn_bar, monus] #|characteristic function of (> or =)
chi_lesser = [comp,chi_greater,[proj,2,2],[proj,1,2]] ## characteristic function of <
chi_greater_or_equal = [comp, chi_lesser_or_equal,[proj,2,2],[proj,1,2]] ## characteristic function of >=

## solution to exercise 5a below
chi_equals = [comp, mul, chi_lesser_or_equal, chi_greater_or_equal ]

#print(arity(pred))
#predProg = print(program(pred))
#onesharp(program(exp),['1#1','11'])
#onesharp(program([comp, pred, [proj,2,2]]), ['11','#1#1'])
#print(successor)

In [ ]:
print(essentially_used(monus))
print(arity(mul))
print(essentially_used([comp, add,[proj, 3,3],[proj, 1, 3]]))
p = program(mul)
parse_explain(p)[240:250]
#onesharp(program(z),['1##1'])
#step_by_step(p, ['#1','1'])
#onesharp(program(monus),['11','##1'])

In [14]:
### exercise 5b below
g = [comp, chi_equals, [proj,2,3], [comp, mul, [proj,1,3],[proj,3,3]]] 
# g tells if first x third = second
#print(arity(g))
#onesharp(program(g),['11','##1','11'])


## an additional function g_2, not part of the exercise 
g2 = [comp, chi_greater, [proj,3,3], [comp, s, [comp, z, [proj, 1, 3]]]]
## g2(m,n,p) =  1 if n > 1, and g2(m,n,p) =  0 if n <= 1
#onesharp(program(g2),['1','#1','#1'])

## now let
g1 = [comp, mul, g, g2]
## g1(m,p,n) = 1 if n is at least 2, and m n = p
#onesharp(program(g1),['1','11','11'])




NameError: ignored

In [32]:
essentially_used(g1)

14

In [ ]:
## exercise 5c below
g3 = [comp, g, [proj, 1, 2], [comp, z, [proj, 1, 2]], [proj, 2, 2]]
p = program(g3)
#onesharp(p, ['#','11111','#111'])
k1 = [comp, add, [proj,4,4], [comp, g, [proj,1,4], [proj,2,4], [comp,s, [proj,3,4]]]]
k = [comp, sgn, k1] 
#print(arity(k))
#print(arity([comp, g, [proj,1,4], [proj,2,4], [comp,[s, [proj,3,4]]))
h = [pr, g3, k]
#print(arity(g3))
#print(arity(h))
#print(arity(k1))
#print(arity(k))

#onesharp(program(h),['11','#11','11'])
## Thus, h does what we want in 5c.
## But we also want to modify h a little to get h1 as on the solutions
## For this, we just use g1 instead of g inside of g3
#g4 = [comp, g1, [proj, 1, 2], [comp, z, [proj, 1, 2]], [proj, 2, 2]]
#k2 = [comp, add, [proj,4,4], [comp, g4, [proj,1,4], [proj,2,4], [comp,s, [proj,3,4]]]]
#k3 = [comp, sgn, k2] 
#h1 = [pr, g4, k3]
#
#print(arity(g4))
#print(arity(k2))
#print(arity(h1))
#onesharp(program(h),['1#1','1#1'])

In [ ]:
# exercise 5d and 5e
# as pointed out by Sam Whitmire,  pd(m,n) = h(m,n, pred(n))
h = [pr, g3, k]
proper_divisor = [comp, h, [proj,1,2], [proj,2,2], [comp, pred, [proj,2,2]]]

#print(arity(proper_divisor))
#onesharp(program(proper_divisor),['11','1##1'])

pd_inverse = [comp, proper_divisor, [proj,2,2],[proj,1,2]]

#onesharp(program(pd_inverse),['1##1','11'])
#onesharp(program([comp, pred, [proj,2,2]]), ['11','#1#1'])

pre_prime = [pr, z, [comp, sgn, [comp, add, [proj,3,3], [comp, pd_inverse, [proj,1,3], [comp, s, [proj,2,3]]]]]]

#arity(pre_prime)
#onesharp(program(pre_prime),['1###1','1###1']) ## gave output 1!!
#onesharp(program(pre_prime),['##11','##11'])
# 6, 5 gives 1
# 7 6 gives #
# 8 7 gives 1
# 9 8 gives 1
# 11 10 gives #
# 12 11 gives 1
# 13 12 gives #
# 12 12 gives 1

# thus, we can do 
prime =[comp, sgn_bar, [comp, pre_prime, [proj,1,1], [comp, pred, [proj,1,1]]]]

onesharp(program(prime),['11#1'])
# Gives output 1 if the input is a prime,
# and gives output # if the output is not a prime.





This is undefined.
The register contents at the end are shown below.


,contents
1,11#1
2,1
3,
4,
5,
6,
7,
8,
9,
10,


In [ ]:
essentially_used(prime)

5

In [117]:
onesharp(program([comp, add,[proj, 3,3],[proj, 1, 3]]),['11','#1','#11'])

This is undefined.
The register contents at the end are shown below.


,contents
1,11
2,#1
3,#11
4,1##1
5,
6,
7,
8,
9,
10,


In [34]:
step_by_step(compare_prog(1,2),['#1','#1'])

First, here is the program:


,instruction,explanation
1,1#####,cases on R1
2,111###,go forward 3 to instruction 5
3,111111###,go forward 6 to instruction 9
4,111111111###,go forward 9 to instruction 13
5,11#####,cases on R2
6,11111111111###,go forward 11 to instruction 17
7,11111111111###,go forward 11 to instruction 18
8,111111111111###,go forward 12 to instruction 20
9,11#####,cases on R2
10,111111111111###,go forward 12 to instruction 22



The computation starts with the register contents shown below.
The registers include those those which you entered as part of the input
and also others mentioned in the input program.


,contents
1,#1
2,#1
3,



Step 1:
Execute instruction 1: cases on R1.
The first symbol in that register is #, so we delete that symbol and go forward 3 instructions.


...

Step 42:
Execute instruction 60: go forward 6 to instruction 66.


,contents
1,1
2,#1#
3,


Step 43:
Execute instruction 66: add 1 to R2.


,contents
1,1
2,#1#1
3,


Step 44:
Execute instruction 67: go forward 15 to instruction 82.
This computation does not halt.
Here is the list of registers whose contents are not empty at this point, other than R1:[2].
The register contents at the end are shown above.


In [35]:
def cmp(dst, src):
  p = ones(dst)+'#####'       # Cases on register ``dst''
  p += '11111 1###'           # Go forward six
  p += '11111 1111###'        # Go forward nine
  p += ones(src)+'#####'      # Cases on register ``src''
  p += '11111 11111 1###'     # Go forward eleven
  p += '11111 11111###'       # Go forward ten
  p += '11111 1####'          # Go backward six
  p += ones(src)+'#####'      # Cases on register ``src''
  p += '11111 11111 11111###' # Go forward fifteen
  p += '11111 1###'           # Go forward six
  p += '11111###'             # Go forward five
  p += ones(src)+'#####'      # Cases on register ``src''
  p += '111###'               # Go forward three
  p += '11111 11111 111####'  # Go backward thirteen
  p += '1###'                 # Go forward one; no op
  p += ones(dst)+'#####'      # Cases on register ``dst''
  p += '111###'               # Go forward three
  p += '11####'               # Go backward two
  p += '111####'              # Go backward three
  p += ones(src)+'#####'      # Cases on register ``src''
  p += '1111###'              # Go forward four
  p += '11####'               # Go backward two
  p += '111####'              # Go backward three
  p += ones(dst)+'#'          # Append ``1'' to register ``dst''
  return p

In [16]:
step_by_step(clear_prog(3),['','1','#'])

First, here is the program:


,instruction,explanation
1,111#####,cases on R3
2,111###,go forward 3 to instruction 5
3,1###,go forward 1 to instruction 4
4,111####,go backward 3 to instruction 1



The computation starts with the register contents shown below.
The registers include those those which you entered as part of the input
and also others mentioned in the input program.


,contents
1,
2,1
3,#



Step 1:
Execute instruction 1: cases on R3.
The first symbol in that register is #, so we delete that symbol and go forward 3 instructions.


...

Step 2:
Execute instruction 4: go backward 3 to instruction 1.


,contents
1,
2,1
3,


Step 3:
Execute instruction 1: cases on R3.
The register is empty, so we go ahead 1 instruction.


,contents
1,
2,1
3,


Step 4:
Execute instruction 2: go forward 3 to instruction 5.
This computation does not halt.
Here is the list of registers whose contents are not empty at this point, other than R1:[2].
The register contents at the end are shown above.


In [69]:

comparison = '1##### 111111### 111111111### 11##### 1111111111 1### 1111111111### 111111#### 11##### 1111111111 11111### 111111### 11111### 11##### 111### 1111111111 111#### 1### 1##### 111### 11#### 111#### 11##### 1111### 11#### 111#### 1#'
comparison.replace(" ","")

'1#####111111###111111111###11#####11111111111###1111111111###111111####11#####111111111111111###111111###11111###11#####111###1111111111111####1###1#####111###11####111####11#####1111###11####111####1#'

In [72]:
onesharp(comparison,['11','11'])
weird = bump(comparison,4)
parse_explain(weird)

,instruction,explanation
1,11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111#,add 1 to R230


In [79]:
comparison.replace(" ","")
def comparison_prog(a,b): #uses a+1, a+2, a+3
  p = copy_prog(a,a+1,a+2) + copy_prog(b,a+2,a+3) +  bump(ddd,a) 
  return(p)

In [86]:
onesharp(comparison_prog(4,2),["1",'####',"","#1##"])

This is undefined.
The register contents at the end are shown below.


,contents
1,1
2,####
3,
4,#1##
5,
6,
7,


In [ ]:
parse_explain(bump)

In [76]:
ddd = comparison.replace(" ","")

In [64]:
comparison

'1##### 111111### 111111111### 11##### 1111111111 1### 1111111111### 111111#### 11##### 1111111111 11111### 111111### 11111### 11##### 111### 1111111111 111#### 1### 1##### 111### 11#### 111#### 11##### 1111### 11#### 111#### 1#'

In [73]:
clear_1

'1#####111###11####111####'

In [78]:
bump(ddd,4)

'11111#####111111###111111111###111111#####11111111111###1111111111###111111####111111#####111111111111111###111111###11111###111111#####111###1111111111111####1###11111#####111###11####111####111111#####1111###11####111####11111#'

In [77]:
ddd

'1#####111111###111111111###11#####11111111111###1111111111###111111####11#####111111111111111###111111###11111###11#####111###1111111111111####1###1#####111###11####111####11#####1111###11####111####1#'

In [92]:
onesharp(compare_prog(4,2),['1','#','#','1'])

This is undefined.
The register contents at the end are shown below.


,contents
1,1
2,#
3,#
4,1
5,
6,
7,


In [8]:
onesharp(compare_prog(3,1),['1##1','1#','1##1',"",""])

This is undefined.
The register contents at the end are shown below.


,contents
1,1##1
2,1#
3,1##1
4,1
5,
6,
